In [ ]:
%pip install flair spacy

In [1]:
import os
os.chdir('/mnt/d/demo')

In [2]:
import torch
print(torch.__version__)

[torch.cuda.get_device_name(d) for d in range(torch.cuda.device_count())]

/home/paudan/.conda/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.11.0


['NVIDIA GeForce RTX 3070 Laptop GPU']

In [3]:
from flair.datasets.sequence_labeling import ColumnCorpus
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from transformers import BertConfig, BertTokenizerFast

In [4]:
BERT_MODEL_DIR = 'FinBERT-FinVocab-Uncased'
CACHE_DIR = 'embeddings'
INPUT_PATH = 'data'
torch.set_default_tensor_type(torch.FloatTensor)

config = BertConfig.from_pretrained(BERT_MODEL_DIR, cache_dir=CACHE_DIR)
config.save_pretrained(BERT_MODEL_DIR)
tokenizer = BertTokenizerFast(vocab_file=os.path.join(BERT_MODEL_DIR, 'vocab.txt'))
tokenizer.save_pretrained(BERT_MODEL_DIR)

('FinBERT-FinVocab-Uncased/tokenizer_config.json',
 'FinBERT-FinVocab-Uncased/special_tokens_map.json',
 'FinBERT-FinVocab-Uncased/vocab.txt',
 'FinBERT-FinVocab-Uncased/added_tokens.json',
 'FinBERT-FinVocab-Uncased/tokenizer.json')

In [5]:
MAX_LENGTH = 512
BATCH_SIZE = 4
MAX_EPOCHS = 5
HIDDEN_SIZE = 256

In [6]:
def filter_long(dataset):
    # Bug in Flair: skip very long sentences to avoid errors!
    return [x for x in dataset if len(x) <= MAX_LENGTH]

def train_tagger(input_path, train_file, test_file, dev_file):
    columns = {0 : 'text', 1 : 'ner'}
    corpus = ColumnCorpus(input_path, columns, train_file=train_file, test_file=test_file, dev_file=dev_file)
    corpus._train = filter_long(corpus.train)
    corpus._dev = filter_long(corpus.dev)
    corpus._test = filter_long(corpus.test)
    embeddings = TransformerWordEmbeddings(BERT_MODEL_DIR, cache_dir=CACHE_DIR, allow_long_sentences=False, fine_tune=True, layers='-1')
    tagger = SequenceTagger(hidden_size=HIDDEN_SIZE, embeddings=embeddings, tag_dictionary=corpus.make_label_dictionary(label_type='ner'), tag_type='ner', use_crf=True)
    trainer = ModelTrainer(tagger, corpus)
    trainer.train('finbert', learning_rate=0.1, mini_batch_size=BATCH_SIZE, max_epochs=MAX_EPOCHS, embeddings_storage_mode='gpu')

In [7]:
train_tagger('data', 'data_train.txt', 'data_test.txt', None)

2022-05-04 15:17:06,229 Reading data from data
2022-05-04 15:17:06,229 Train: data/data_train.txt
2022-05-04 15:17:06,230 Dev: None
2022-05-04 15:17:06,230 Test: data/data_test.txt
2022-05-04 15:17:11,896 Computing label dictionary. Progress:


1046it [00:00, 95009.25it/s]

2022-05-04 15:17:11,910 Dictionary created for label 'ner' with 5 values: PERSON (seen 661 times), ORG (seen 230 times), LOC (seen 162 times), MISC (seen 7 times)
2022-05-04 15:17:11,911 SequenceTagger predicts: Dictionary with 17 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-ORG, B-ORG, E-ORG, I-ORG, S-LOC, B-LOC, E-LOC, I-LOC, S-MISC, B-MISC, E-MISC, I-MISC


2022-05-04 15:17:12,568 ----------------------------------------------------------------------------------------------------
2022-05-04 15:17:12,570 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30873, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): D

100%|██████████| 29/29 [00:01<00:00, 19.40it/s]

...

100%|██████████| 29/29 [00:01<00:00, 18.83it/s]

2022-05-04 15:21:39,479 Evaluating as a multi-label problem: False
2022-05-04 15:21:39,484 DEV : loss 0.14075566828250885 - f1-score (micro avg)  0.0
2022-05-04 15:21:39,489 BAD EPOCHS (no improvement): 1


2022-05-04 15:21:41,262 ----------------------------------------------------------------------------------------------------
2022-05-04 15:21:41,265 loading file finbert/best-model.pt
2022-05-04 15:23:10,072 SequenceTagger predicts: Dictionary with 19 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-ORG, B-ORG, E-ORG, I-ORG, S-LOC, B-LOC, E-LOC, I-LOC, S-MISC, B-MISC, E-MISC, I-MISC, <START>, <STOP>


100%|██████████| 75/75 [00:04<00:00, 15.60it/s]

2022-05-04 15:23:16,177 Evaluating as a multi-label problem: False
2022-05-04 15:23:16,184 0.6	0.594	0.597	0.455
2022-05-04 15:23:16,185 
Results:
- F-score (micro) 0.597
- F-score (macro) 0.1867
- Accuracy 0.455

By class:
              precision    recall  f1-score   support

      PERSON     0.6390    0.8985    0.7468       197
         ORG     0.0000    0.0000    0.0000        56
         LOC     0.0000    0.0000    0.0000        39
        MISC     0.0000    0.0000    0.0000         6

   micro avg     0.6000    0.5940    0.5970       298
   macro avg     0.1597    0.2246    0.1867       298
weighted avg     0.4224    0.5940    0.4937       298

2022-05-04 15:23:16,186 ----------------------------------------------------------------------------------------------------
